# **Before each experiment**

Select `Kernel` > `Restart Kernel and Clear All Outputs`

In [ ]:
%%cmd
poetry update

***
# **Without mouse on stage**

In [32]:
import datetime
import pathlib
import time

import IPython
import ipywidgets as widgets

import np_config
import np_logging
import np_services
import np_session
import np_workflows
from np_probe_targets.implant_drawing import DRWeeklyTargets

from np_services import (
    Sync, VideoMVR, ImageMVR, ScriptCamstim, NewScaleCoordinateRecorder, OpenEphys,
)
from np_services.protocols import (
    Pretestable, Initializable, Startable, Testable, Stoppable, Finalizable, Pretestable, Verifiable, Validatable,
    TestError,
    )

logger = np_logging.getLogger()

np_services.start_rsc_apps()

np_workflows.elapsed_time_widget()

***
## "Quiet mode"
on  
*default*
- exception details are hidden
- regular messages displayed (log level = INFO)

off
- full exception details with traceback
- extra messages displayed (log level = DEBUG)

In [ ]:
np_workflows.quiet_mode_widget()

***
## Select mouse and user

In [ ]:
user, mouse = np_workflows.user_and_mouse_widget()

***
## Check MTrain for mouse
[optional]

In [ ]:
np_workflows.mtrain_widget(mouse)

***
## Check task

In [ ]:
task_name = 'stage 5 AMN ori moving'

***
## Generate new session and folder 

In [ ]:
# session = np_session.generate_session(mouse, user, session_type='ecephys')

data_root = pathlib.Path('Y://')

session_folder = data_root / f'{datetime.datetime.now():%Y-%m-%d_%H-%M-%S}_{mouse}'
session_folder.mkdir(parents=True, exist_ok=True)

***
## Pretest
Checks before running

In [ ]:
np_workflows.check_hardware_widget()

In [ ]:
np_workflows.check_openephys_widget()

## Test critical components individually
Only proceed if all pretests pass!

In [ ]:
OpenEphys.pretest()

In [ ]:
VideoMVR.pretest()

In [ ]:
ImageMVR.pretest()

In [ ]:
Sync.pretest()

In [ ]:
ScriptCamstim.pretest()

***
## Setup and reset all services

In [ ]:
ScriptCamstim.script = '//allen/programs/mindscope/workgroups/dynamicrouting/DynamicRoutingTask/runTask.py'

task_params = dict(
    rigName = 'NP3',
    subjectName = str(mouse),
    taskScript = '//allen/programs/mindscope/workgroups/dynamicrouting/DynamicRoutingTask/DynamicRouting1.py',
    taskVersion = task_name,
)

mapping_params = dict(
    rigName = 'NP3',
    subjectName = str(mouse),
    taskScript = '//allen/programs/mindscope/workgroups/dynamicrouting/DynamicRoutingTask/RFMapping.py'
)

sound_test_params = dict(
    rigName = 'NP3',
    subjectName = 'sound',
    taskScript = '//allen/programs/mindscope/workgroups/dynamicrouting/DynamicRoutingTask/TaskControl.py',
    taskVersion = 'sound test',
)

ScriptCamstim.data_root = pathlib.Path('//allen/programs/mindscope/workgroups/dynamicrouting/DynamicRoutingTask/Data') / str(mouse)

OpenEphys.folder = session_folder.name

NewScaleCoordinateRecorder.log_root = session_folder

services = (Sync, VideoMVR, ImageMVR, OpenEphys, NewScaleCoordinateRecorder, ScriptCamstim)

for service in services:
    if isinstance(service, Initializable):
        service.initialize()
    if isinstance(service, Testable):
        service.test()

***
## Run sound test

In [ ]:
ScriptCamstim.params = sound_test_params
ScriptCamstim.start()

while not ScriptCamstim.is_ready_to_start():
    time.sleep(1)

# re-initialize in case sound test produces pkl files that we don't want:
ScriptCamstim.initialize()

***
## Dip probes

In [ ]:
np_workflows.dye_widget(session_folder)

In [ ]:
np_workflows.photodoc_widget('pre_experiment_surface_image')

***
***
# **With mouse on stage**

In [ ]:
np_workflows.check_mouse_widget()

***
## Lower cartridge

In [ ]:
np_workflows.photodoc_widget('brain_surface_image')

***
## Insert probes

In [ ]:
np_workflows.isi_widget(mouse.lims, colormap=False)
np_workflows.probe_targeting_widget(session_folder)

In [35]:
DRWeeklyTargets(2)

***
## Probes at surface, before advancing

In [ ]:
np_workflows.photodoc_widget('pre_insertion_surface_image')

***
## Probes at target depth

In [ ]:
np_workflows.photodoc_widget('post_insertion_surface_image')

***
## Settle timer

In [ ]:
np_workflows.print_countdown_timer(minutes=20)

***
## Start devices recording

In [ ]:
recorders = (Sync, VideoMVR, OpenEphys)
with np_services.stop_on_error(*recorders):
    for service in recorders:
        service.start()
        if isinstance(service, Verifiable):
            time.sleep(2)
            service.verify()

***
## Start mapping

In [ ]:
ScriptCamstim.params = mapping_params 

logger.info('Starting mapping')

ScriptCamstim.start()

# time.sleep(10)
# ScriptCamstim.verify() # needs updating to not check filesize

while not ScriptCamstim.is_ready_to_start():
    time.sleep(10)

if isinstance(ScriptCamstim, Finalizable):
    ScriptCamstim.finalize()

logger.info('Mapping complete')

***
Give mousey a reward to test water delivery
***
## Start task

In [ ]:
ScriptCamstim.params = task_params

logger.info('Starting task')

ScriptCamstim.start()

while not ScriptCamstim.is_ready_to_start():
    time.sleep(10)

if isinstance(ScriptCamstim, Finalizable):
    ScriptCamstim.finalize()

***
## Stop recording

In [ ]:
while not ScriptCamstim.is_ready_to_start():
    time.sleep(10)

for service in (_ for _ in (Sync, VideoMVR, OpenEphys) if isinstance(_, Stoppable)):
    service.stop()

for service in (_ for _ in (VideoMVR, OpenEphys, ScriptCamstim) if isinstance(_, Finalizable)): # Sync will be finalized later to avoid waiting on processing
    service.finalize()

for service in (_ for _ in (VideoMVR, OpenEphys, ScriptCamstim) if isinstance(_, Validatable)):
    service.validate()

***
## Before removing probes

In [ ]:
np_workflows.photodoc_widget('post_stimulus_surface_image')

***
## After fully retracting probes

In [ ]:
np_workflows.photodoc_widget('post_experiment_surface_image')

***
## After raising cartridge

In [ ]:
np_workflows.finishing_checks_widget()
np_workflows.copy_files(services, session_folder)